In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Practica - Twitter
# ---------------------------------------------------
# Librerias necesarias para la realizacion de la practica:

# Instalacion:
install.packages("rtweet")
install.packages("tidyverse")
install.packages("tidytext")
install.packages("knitr")
install.packages("workcloud")
install.packages("igraph")
install.packages("ggraph")
install.packages("quanteda")
install.packages("e1071")
install.packages("SparseM")
install.packages("RSelenium")
install.packages("sm")
install.packages("wordcloud")
install.packages("RColorBrewer")
install.packages("hrbrthemes")
install.packages("UpSetR")
install.packages("eechidna")
install.packages("cartogram")

# Carga:
library(rtweet)
library(tidyverse)
library(tidytext)
library(knitr)
library(workcloud)
library(igraph)
library(ggraph)
library(quanteda)
library(e1071)
library(SparseM)
library(RSelenium)

library(scales)
library(gridExtra)
library(tm)
library(lubridate)
library(zoo)
library(wordcloud)
library(RColorBrewer)
library(hrbrthemes)
library(UpSetR)
library(eechidna)
library(cartogram)

tema_graf <-
  theme_minimal() +
  theme(text = element_text(family = "serif"),
        panel.grid.minor = element_blank(),
        strip.background = element_rect(fill = "#EBEBEB", colour = NA),
        legend.position = "none",
        legend.box.background = element_rect(fill = "#EBEBEB", colour = NA))
  
# ---------------------------------------------------
# Extracción de información en twitter:

# 1) Creación de token con las claves en el API de Twitter
token <- create_token(
app = "U5_Practica_Ana",
consumer_key = "I3vOd8YnGdjCHLixAv9MnrZfg",
consumer_secret = "AFGGNIqtuMHzH6QBVXPw84UOYxwaQdmIHET6tcbX1YprZvbxYw",
access_token = "712134627888992256-B6Vbp2bEQ0LjwTWksE6oHXWVxRjLch5",
access_secret = "X00rSqIdvec8yA0yufnAgf3AVCSsLMgxTOutbF7cIHnc7"
)

# 2) Extracción de tweets de tres 
# usuarios mediante la función get_timeline
datos_new1 <- get_timeline(user="@LuisitoComunica", n = 200, parse = TRUE, check = TRUE, include_rts = FALSE)
datos_new2 <- get_timeline(user="@franco_esca", n = 200, parse = TRUE, check = TRUE, include_rts = FALSE)
datos_new3 <- get_timeline(user="@MYMALK4PON3", n = 200, parse = TRUE, check = TRUE, include_rts = FALSE)

# 3) Conversión de columnas numéricas a character, para evitar 
# errores de compatibilidad, mediante la función map_if
datos_new1 <- map_if(.x=datos_new1, .p=is.numeric, .f=as.character)
datos_new2 <- map_if(.x=datos_new2, .p=is.numeric, .f=as.character)
datos_new3 <- map_if(.x=datos_new3, .p=is.numeric, .f=as.character)

# 4) Unión de todos los tweets en un único dataframe
tweets1 <- bind_rows(datos_new1, datos_new2, datos_new3)

# 5) Mostrar el total de tweets extraídos 
resumen <- tweets1 %>% group_by("screen_name") %>% summarise(numero_tweets = n()); resumen

# 6) Seleccionar  y renombrar columnas
tweets1 <- tweets1 %>% select(screen_name, created_at, status_id, text)
tweets1 <- tweets1 %>% rename(autor = screen_name, fecha = created_at, texto = text, tweet_id = status_id)
head(tweets1)
colnames(tweets1)

# ---------------------------------------------------
# Limpieza de datos y tokenización:

# crear script  con el siguiente codigo, y guardarlo. 
# El script, para cada tweet:
# 1) cambia el texto a minúsculas elimina 
# 2) páginas web
# 3) signos de puntuación
# 4) números
# 5) espacios en blanco seguidos
# 6) separa las palabras con más de 1 caracter en tokens

limpiar_tokenizar <- function(texto){
nuevo_texto <- tolower(texto) # 1
# de cualquier cosa que no sea un espacio)
nuevo_texto <- str_replace_all(nuevo_texto,"http\\S*", "") # 2
nuevo_texto <- str_replace_all(nuevo_texto,"[[:punct:]]", " ") # 3
nuevo_texto <- str_replace_all(nuevo_texto,"[[:digit:]]", " ") # 4
nuevo_texto <- str_replace_all(nuevo_texto,"[\\s]+", " ")  # 5
nuevo_texto <- str_split(nuevo_texto, " ")[[1]]   #6
nuevo_texto <- keep(.x = nuevo_texto, .p = function(x){str_length(x) > 1})
    return(nuevo_texto)
}

# 1) Se aplica la función de limpieza y 
# tokenización a cada tweet
tweets1 <- tweets1 %>% mutate(texto_tokenizado = map(.x = texto, .f = limpiar_tokenizar))
tweets1 %>% select(texto_tokenizado) %>% head()

In [ ]:
# Practica - Twitter
# ---------------------------------------------------
# Librerias necesarias para la realizacion de la practica:

# Instalacion:
install.packages("rtweet")
install.packages("tidyverse")
install.packages("tidytext")
install.packages("knitr")
#install.packages("workcloud")
install.packages("igraph")
install.packages("ggraph")
install.packages("quanteda")
install.packages("e1071")
install.packages("SparseM")
install.packages("RSelenium")
install.packages("sm")
install.packages("wordcloud")
install.packages("RColorBrewer")
install.packages("hrbrthemes")
install.packages("UpSetR")
install.packages("eechidna")
install.packages("cartogram")
install.packages("twitteR")
install.packages("ROAuth")

In [ ]:
# Carga:
library(rtweet)
library(tidyverse)
library(tidytext)
library(knitr)
#library(workcloud)
library(igraph)
library(ggraph)
library(quanteda)
library(e1071)
library(SparseM)
library(RSelenium)

library(scales)
library(gridExtra)
library(tm)
library(lubridate)
library(zoo)
library(wordcloud)
library(RColorBrewer)
library(hrbrthemes)
library(UpSetR)
library(eechidna)
library(cartogram)
library(twitteR)
library(ROAuth)

In [ ]:
#CONFIGURACIÓN
tema_graf <-
  theme_minimal() +
  theme(text = element_text(family = "serif"),
        panel.grid.minor = element_blank(),
        strip.background = element_rect(fill = "#EBEBEB", colour = NA),
        legend.position = "none",
        legend.box.background = element_rect(fill = "#EBEBEB", colour = NA))

In [ ]:
# 1) Creación de token con las claves en el API de Twitter
app = "BigDataAcevedo"
consumer_key = "hjB4PfLczGIDn85fWXqeAAAFX"
consumer_secret = "AjRlhWBC2kgbnVnzjRkZQLlAN8x18BOxKT5ZDEuLIGosB9M0ed"
access_token = "1388188187433054209-p96F7Jj56cN35NDD1stz6MSVLF8ph9"
access_secret = "7EcdTVdpL72nRE7JscNOTurYJt2XFgivGpK0BvhJvpWCe"

setup_twitter_oauth(consumer_key,consumer_secret,access_token,access_secret)


In [ ]:
# 2) Extracción de tweets de tres 
cuenta1 <- userTimeline("@vegetta777", n = 45)
cuenta1
cuenta2 <- userTimeline("@Rubiu5", n=45)
cuenta2
cuenta3 <- userTimeline("@WillyrexYT", n=45)
cuenta3 
typeof(cuenta1) 
typeof(cuenta2)
typeof(cuenta3)
# @Rubiu5 @JeltyTV @ElBadiablo @franco_esca @JuanSGuarnizo


In [ ]:
# 3) Conversión de columnas numéricas a character, 
#para evitar errores de compatibilidad, mediante la función map_if
cuenta1 <- map_if(.x=cuenta1, .p=is.numeric, .f=as.character)
cuenta1 = twListToDF(cuenta1)

cuenta2 <- map_if(.x=cuenta2, .p=is.numeric, .f=as.character)
cuenta2 = twListToDF(cuenta2)

cuenta3 <- map_if(.x=cuenta3, .p=is.numeric, .f=as.character)
cuenta3 = twListToDF(cuenta3)


In [ ]:
# 4) Unión de todos los tweets en un único dataframe
tweets1 <- rbind(cuenta1, cuenta2, cuenta3)
tweets1
write.csv(data, "Streamers.csv")

In [ ]:
# 5) Mostrar el total de tweets extraídos 
resumen <- tweets1 %>% group_by("screenName") %>% summarise(numero_tweets = n()); resumen

In [ ]:
# 6) Seleccionar  y renombrar columnas
tweets1 <- tweets1 %>% select(screenName, created, id, text)
tweets1 <- tweets1 %>% rename(autor = screenName, fecha = created, texto = text, tweet_id = id)
head(tweets1)
colnames(tweets1)

In [ ]:
#7 Tokeniza - Separa las cadenas en palabras
limpiar_tokenizar <- function(texto){
nuevo_texto <- tolower(texto) # 1
# de cualquier cosa que no sea un espacio)
nuevo_texto <- str_replace_all(nuevo_texto,"http\\S*", "") # 2
nuevo_texto <- str_replace_all(nuevo_texto,"[[:punct:]]", " ") # 3
nuevo_texto <- str_replace_all(nuevo_texto,"[[:digit:]]", " ") # 4
nuevo_texto <- str_replace_all(nuevo_texto,"[\\s]+", " ")  # 5
 nuevo_texto <- str_replace_all(nuevo_texto,"[[emoji:]]+", " ")   
nuevo_texto <- str_split(nuevo_texto, " ")[[1]]   #6
nuevo_texto <- keep(.x = nuevo_texto, .p = function(x){str_length(x) > 1})
    return(nuevo_texto)
}


In [ ]:
tweets1 <- tweets1 %>% mutate(texto_tokenizado = map(.x = texto, .f = limpiar_tokenizar))
tweets1 %>% select(texto_tokenizado) %>% head()

In [ ]:
#--------- 1) Al tokenizar, el elemento de estudio ha pasado a 
# ser cada token, incumpliendo así la condición de tidy data. 

tweets_tidy <- tweets1 %>% select(-texto) %>% unnest()
tweets_tidy <- tweets_tidy %>% rename(token = texto_tokenizado)
head(tweets_tidy) 

In [ ]:
# 2) Crear una gráfica de la distribución de los tweets 
# recuperados en función de la fecha en que fueron publicados
ggplot(tweets1, aes(x = as.Date(fecha), fill = autor)) +
  geom_histogram(position = "identity", bins = 20, show.legend = FALSE) +
  scale_x_date(date_labels = "%m-%Y", date_breaks = "2 week") +
  labs(x = "fecha de publicación", y = "número de tweets") +
  facet_wrap(~ autor, ncol = 1) +  theme_bw() +  theme(axis.text.x = element_text(angle = 90))

tweets_mes_anyo <- tweets1 %>% mutate(mes_anyo = format(fecha, "%Y-%m"))
tweets_mes_anyo %>% group_by(autor, mes_anyo) %>% summarise(n = n()) %>%  ggplot(aes(x = mes_anyo, y = n, color = autor)) +
  geom_line(aes(group = autor)) +
  labs(title = "Número de tweets publicados", x = "fecha de publicación",
       y = "número de tweets") +  theme_bw() +
  theme(axis.text.x = element_text(angle = 90, size = 6),
        legend.position = "bottom")

In [ ]:
# 3) Conteo de palabras por usuario: 
tweets_tidy %>% group_by(autor) %>% summarise(n = n()) 
tweets_tidy %>%  ggplot(aes(x = autor)) + geom_bar() + coord_flip() + theme_bw()

In [ ]:
# 4) onteo de palabras distintas por usuario:
tweets_tidy %>% select(autor, token) %>% distinct() %>%  group_by(autor) %>%  summarise(palabras_distintas = n()) 
tweets_tidy %>% select(autor, token) %>% distinct() %>%
            ggplot(aes(x = autor)) + geom_bar() + coord_flip() + theme_bw()

In [ ]:
# 8) Relación entre palabras (limpieza de tweets)
limpiar <- function(texto){
# Se convierte todo el texto a minúsculas
    nuevo_texto <- tolower(texto)
    # Eliminación de páginas web (palabras que empiezan por "http." seguidas 
    # de cualquier cosa que no sea un espacio)
    nuevo_texto <- str_replace_all(nuevo_texto,"http\\S*", "")
    # Eliminación de signos de puntuación
    nuevo_texto <- str_replace_all(nuevo_texto,"[[:punct:]]", " ")
    # Eliminación de números
    nuevo_texto <- str_replace_all(nuevo_texto,"[[:digit:]]", " ")
    # Eliminación de espacios en blanco múltiples
    nuevo_texto <- str_replace_all(nuevo_texto,"[\\s]+", " ")
    return(nuevo_texto)
}

In [ ]:
# 9) Stopwords (palabras que no aportan al texto)
require(sm)
lista_stopwords <- stopwords("spanish")
# Se añade el término amp al listado de stopwords
lista_stopwords <- c(lista_stopwords, "amp")

# Se filtran las stopwords
tweets_tidy <- tweets_tidy %>% filter(!(token %in% lista_stopwords))
lista_stopwords

In [ ]:
#5) Correlación entre usuarios por palabras utilizadas:
tweets_spread <- tweets_tidy %>% group_by(autor, token) %>% count(token) %>%
                 spread(key = autor, value = n, fill = NA, drop = TRUE)
cor.test(~ Rubiu5 + ElBadiablo, method = "pearson", data = tweets_spread)

p1 <- ggplot(tweets_spread, aes(Rubiu5, JuanSGuarnizo)) + geom_jitter(alpha = 0.1, size = 2.5, width = 0.25, height = 0.25) +      geom_text(aes(label = token), check_overlap = TRUE, vjust = 1.5) + 
      scale_x_log10(labels = percent_format()) + scale_y_log10(labels = percent_format()) +
      geom_abline(color = "red") + theme_bw() + theme(axis.text.x = element_blank(),
            axis.text.y = element_blank())

p2 <- ggplot(tweets_spread, aes(ElBadiablo, JuanSGuarnizo)) + geom_jitter(alpha = 0.1, size = 2.5, width = 0.25, height = 0.25) +      geom_text(aes(label = token), check_overlap = TRUE, vjust = 1.5) + 
      scale_x_log10(labels = percent_format()) + scale_y_log10(labels = percent_format()) +
      geom_abline(color = "red") + theme_bw() + theme(axis.text.x = element_blank(),
            axis.text.y = element_blank())

grid.arrange(p1, p2, nrow = 1)
# @Rubiu5 @JeltyTV @ElBadiablo @franco_esca @JuanSGuarnizo

In [ ]:
# 6) Conteo de palabras comunes entre dos usuarios: 
palabras_comunes <- dplyr::intersect(tweets_tidy %>% filter(autor=="JuanSGuarnizo") %>%
                    select(token), tweets_tidy %>% filter(autor=="Rubiu5") %>%
                    select(token)) %>% nrow()
paste("Número de palabras comunes entre JuanSGuarnizo y Rubiu5: ", palabras_comunes)

palabras_comunes <- dplyr::intersect(tweets_tidy %>% filter(autor=="JuanSGuarnizo") %>%
                    select(token), tweets_tidy %>% filter(autor=="ElBadiablo") %>%
                    select(token)) %>% nrow()
paste("Número de palabras comunes entre JuanSGuarnizo y ElBadiablo: ", palabras_comunes)

In [ ]:
# 7) Comparación en el uso de palabras:
# a) Pivotaje y despivotaje
tweets_spread <- tweets_tidy %>% group_by(autor, token) %>% count(token) %>% spread(key = autor, value = n, fill = 0, drop = TRUE)
tweets_unpivot <- tweets_spread %>% gather(key = "autor", value = "n", -token)

# b) Selección de los autores JuanSGuarnizo y Rubiu5
tweets_unpivot <- tweets_unpivot %>% 
filter(autor %in% c("Rubiu5", "JuanSGuarnizo"))

# c) Se añade el total de palabras de cada autor
tweets_unpivot <- tweets_unpivot %>% left_join(tweets_tidy %>% group_by(autor) %>%summarise(N = n()), by = "autor")
tweets_unpivot

# d) Cálculo de odds y log of odds de cada palabra
tweets_logOdds <- tweets_unpivot %>%  mutate(odds = (n + 1) / (N + 1))
tweets_logOdds <- tweets_logOdds %>% select(autor, token, odds) %>% spread(key = autor, value = odds)
tweets_logOdds <- tweets_logOdds %>% mutate(log_odds = log(Rubiu5/ElBadiablo), abs_log_odds = abs(log_odds))
tweets_logOdds <- tweets_logOdds %>% mutate(autor_frecuente = if_else(log_odds > 0, "@Rubiu5", "@ElBadiablo"))
tweets_logOdds %>% arrange(desc(abs_log_odds)) %>% head() 
a <- tweets_logOdds %>% group_by(autor_frecuente) %>% top_n(15, abs_log_odds)

# f) Graficar resultados
ggplot(a,aes(x = reorder(token, log_odds), y = log_odds, fill = autor_frecuente)) + 
geom_col() + labs(x = "palabra", y = "log odds ratio (Rubiu5 / ElBadiablo)") + coord_flip() +  theme_bw()


#Representación gráfica de las frecuencias
tweets_tidy %>% group_by(autor, token) %>% count(token) %>% group_by(autor) %>%
                top_n(10, n) %>% arrange(autor, desc(n)) %>%
                ggplot(aes(x = reorder(token,n), y = n, fill = autor)) +
                geom_col() +
                theme_bw() +
                labs(y = "", x = "") +
                theme(legend.position = "none") +
                coord_flip() +
                facet_wrap(~autor,scales = "free", ncol = 1, drop = TRUE)

In [ ]:
# Practica 2 - Clasificación de tweets:

# Para poder aplicar algoritmos de clasificación 
# a un texto, es necesario crear una representación 
# numérica del mismo
tweets_rubi_juan <- tweets1 %>% filter(autor %in% c("Rubiu5", "JuanSGuarnizo"))
set.seed(123)
train <- sample(x = 1:nrow(tweets_rubi_juan), size= 0.8*nrow(tweets_rubi_juan))
tweets_train <- tweets_rubi_juan[train, ]
tweets_test  <- tweets_rubi_juan[-train, ]
# @Rubiu5  @ElBadiablo  @JuanSGuarnizo

In [ ]:
# Es importante verificar que la proporción de 
# cada grupo es similar en el set de entrenamiento 
# y en el de test.
table(tweets_train$autor) / length(tweets_train$autor)
table(tweets_test$autor) / length(tweets_test$autor)

# Limpieza y tokenización de los documentos de entrenamiento
tweets_train$texto <- tweets_train$texto %>% map(.f = limpiar_tokenizar) %>% map(.f = paste, collapse = " ") %>% unlist()

In [ ]:
# Creación de la matriz documento-término
matriz_tfidf_train <- dfm(x = tweets_train$texto, remove = lista_stopwords)

# Se reduce la dimensión de la matriz eliminando aquellos 
# términos que aparecen en menos de 5 documentos. Con esto se consigue eliminar ruido.
matriz_tfidf_train <- dfm_trim(x = matriz_tfidf_train, min_docfreq = 5)

In [ ]:
# Conversión de los valores de la matriz a tf-idf
matriz_tfidf_train <- dfm_tfidf(matriz_tfidf_train, scheme_tf = "prop",scheme_df = "inverse")
matriz_tfidf_train

In [ ]:
# Limpieza y tokenización de los documentos de test
tweets_test$texto <- tweets_test$texto %>% map(.f = limpiar_tokenizar) %>%map(.f = paste, collapse = " ") %>% unlist()

In [ ]:
# Identificación de las dimensiones de la matriz de entrenamiento
# Los objetos dm() son de clase S4, se accede a sus elementos mediante @
dimensiones_matriz_train <- matriz_tfidf_train@Dimnames$features

In [ ]:
# Conversión de vector a diccionario pasando por lista
dimensiones_matriz_train <- as.list(dimensiones_matriz_train)
names(dimensiones_matriz_train) <- unlist(dimensiones_matriz_train)
dimensiones_matriz_train <- dictionary(dimensiones_matriz_train)
dimensiones_matriz_train

In [ ]:
# Proyección de los documentos de test
matriz_tfidf_test<-dfm(x=tweets_test$texto,dictionary=dimensiones_matriz_train)
matriz_tfidf_test<-dfm_tfidf(matriz_tfidf_test, scheme_tf="prop",scheme_df = "inverse")
matriz_tfidf_test

In [ ]:
# Como modelo de predicción se emplea una Máquina de soporte vectorial (SVM). 
# Este método de aprendizaje estadístico suele dar buenos resultados en clasificación.
# Ajuste del modelo
modelo_svm <- svm(x = matriz_tfidf_train, y = as.factor(tweets_train$autor),
                  kernel = "linear", cost = 1, scale = TRUE, type = "C-classification")
modelo_svm

In [ ]:
# Predicciones
predicciones <- predict(object = modelo_svm, newdata = matriz_tfidf_test)

# Error de predicción
# Matriz de confusión
table(observado = tweets_test$autor, predicho = predicciones)

In [ ]:
# fuente: https://rpubs.com/jboscomendoza/analisis_sentimientos_lexico_afinn
download.file("https://raw.githubusercontent.com/jboscomendoza/rpubs/master/sentimientos_afinn/lexico_afinn.en.es.csv",
              "lexico_afinn.en.es.csv")
sentimientos <- read.csv("lexico_afinn.en.es.csv", stringsAsFactors = F, fileEncoding = "latin1") %>% 
  tbl_df()
sentimientos

In [ ]:
# 1) En este ejercicio se emplea la clasificación 
# positivo/negativo proporcionada por el diccionario bing.

sentimientos <- get_sentiments(lexicon = "bing")
# head(sentimientos)
sentimientos <- sentimientos %>% mutate(valor = if_else(sentiment == "negative", -1, 1))
sentimientos

In [ ]:
# Sentimiento promedio de cada tweet
# Al disponer de los datos en formato tidy 
# (una palabra por fila), mediante un inner join 
# se añade a cada palabra su sentimiento y 
# se filtran automáticamente todas aquellas 
# palabras para las que no hay información disponible.
tweets_sent <- inner_join.data.frame(x = tweets_tidy, y = sentimientos, by = c(token = "palabra")))
tweets_sent